In [2]:
import numpy as np
from numpy import pi
from ssid import okid, ExtractModes
from control.matlab import ss
from control.matlab import impulse as _impulse
def impulse(*args, **kwds):
    a,t = _impulse(*args, **kwds)
    return a.squeeze(),t 
from scipy import signal
from scipy.signal import dimpulse as _dimpulse
def dimpulse(*args, **kwds):
    t,a = _dimpulse(*args, **kwds)
    return [ai.squeeze() for ai in a],t 
from matplotlib import pyplot as plt
from collections import defaultdict


# Real Time History Data

In [3]:
# See the real response data to test

import pickle
from datetime import datetime
from CE58658.utilities import husid

with open('../../CalTrans.Hayward/Studies/rawRHs.dat', 'rb') as f:
    rawRHs = pickle.load(f)

def plotSensorRH(rawRHs, event_date, channel_numbers, dt, windowed=False):   
    event = rawRHs[event_date]

    accel_unit_label = r"$\ddot{u}$ [g]"
    accel_scale = 1/980.665
    displ_unit_label = r"$u$ [in]"
    displ_scale = 1/2.54

    fig, ax = plt.subplots(len(channel_numbers), 2, figsize=(12,1.75*len(channel_numbers)), constrained_layout=True)
    if dt is not None:
        x_units = "s"
    else:
        dt = 1.0
        x_units = "timestep"
    sensorRH0 = event['24']['accel']*accel_scale
    times = dt*np.arange(len(sensorRH0))
    if windowed:
        window = husid(sensorRH0, False, dt, lb=0.025, ub=0.975)
        times = times[window[0]:window[1]]
    else:
        window = [0,len(sensorRH0)]
        fig, ax = plt.subplots(1,2,figsize=(15,3))

    accels = defaultdict(dict)
    for i,nm in enumerate(channel_numbers):
        accel_series = event[nm]['accel'][window[0]:window[1]]*accel_scale
        accels[nm] = event[nm]['accel']*accel_scale
        displ_series = event[nm]['displ'][window[0]:window[1]]*displ_scale
        direction = event[nm]['direction']
        channel_name = event[nm]['channel_name']
        ax[i,0].plot(times, accel_series, linewidth=0.75)
        ax[i,0].set_ylabel(f"{accel_unit_label}")
        ax[i,0].set_title(f"Channel {nm} ({channel_name}) ({direction})")
        ax[i,1].plot(times, displ_series, linewidth=0.75)
        ax[i,1].set_ylabel(f"{displ_unit_label}")
        ax[i,1].set_title(f"Channel {nm} ({channel_name}) ({direction})")
    ax[-1,0].set_xlabel(f"time [{x_units}]")
    ax[-1,1].set_xlabel(f"time [{x_units}]")
    fig.suptitle(f"Response History of {datetime.strptime(event_date, '%Y-%m-%dT%H:%M:%S')}")
    fig.align_labels()
    plt.show()
    return accels

def getSensorRH(rawRHs, event_date, channel_numbers):   
    event = rawRHs[event_date]
    accel_scale = 1/980.665
    accels = defaultdict(dict)
    for nm in channel_numbers:
        accels[nm] = event[nm]['accel']*accel_scale
    return accels

CHANNELS = {
# channel node dof rotationangle locationname
    "1": (1031, 3, 37.66*pi/180, "abutment_1"),
    "2": (1031, 2, 37.66*pi/180, "abutment_1"),
    "3": (1031, 1, 37.66*pi/180, "abutment_1"),
    "6": (307, 1, 31.02*pi/180, "bent_3_south_column_grnd_level"),
    "7": (307, 2, 31.02*pi/180, "bent_3_south_column_grnd_level"),
    "11": (1030, 3, 37.66*pi/180, "deck_level_near_abut_1"),
    "12": (1030, 1, 37.66*pi/180, "deck_level_near_abut_1"),
    "13": (1030, 2, 37.66*pi/180, "deck_level_near_abut_1"),
    "14": (304, 1, 31.02*pi/180, "bent_3_deck_level"),
    "15": (304, 2, 31.02*pi/180, "bent_3_deck_level"),
    # "16": (30003, 3, ((31.02+26.26)/2)*pi/180, "midspan_between_bents_3_4_deck"),
    "17": (401, 1, 26.26*pi/180, "bent_4_north_column_grnd_level"),
    "18": (401, 2, 26.26*pi/180, "bent_4_north_column_grnd_level"),
    "19": (402, 1, 26.26*pi/180, "bent_4_north_column_top"),
    "20": (402, 2, 26.26*pi/180, "bent_4_north_column_top"),
    "21": (405, 3, 26.26*pi/180, "bent_4_deck_level"),
    "22": (405, 1, 26.26*pi/180, "bent_4_deck_level"),
    "23": (405, 2, 26.26*pi/180, "bent_4_deck_level"),
    "24": (407, 1, 26.26*pi/180, "bent_4_south_column_grnd_level"),
    "25": (407, 2, 26.26*pi/180, "bent_4_south_column_grnd_level")
}

NODE_INFO = {
# key is node number, value is ((ch1,ch2,ch3), location name, rotation angle).
    '1031':  ((3,2,1),        'abutment_1',                      -37.66*pi/180),
    '1030':  ((12,13,11),     'deck_level_near_abut_1',          -37.66*pi/180),
    '304':   ((14,15,None),   'bent_3_deck_level',               -31.02*pi/180),
    '307':   ((6,7,None),     'bent_3_south_column_grnd_level',  -31.02*pi/180),
    '30003': ((None,None,16), 'midspan_between_bents_3_4_deck',  -((31.02+26.26)/2)*pi/180), 
    '401':   ((17,18,None),   'bent_4_north_column_grnd_level',  -26.26*pi/180),
    '402':   ((19,20,None),   'bent_4_north_column_top',         -26.26*pi/180),
    '405':   ((22,23,None),   'bent_4_deck_level',               -26.26*pi/180),
    '407':   ((24,25,None),   'bent_4_south_column_grnd_level',  -26.26*pi/180)
}

channel_numbers = CHANNELS.keys()
channel_numbers = ['12', '13', '14', '15', '18', '19', '20', '22', '23']
output_channel_numbers = ['12', '13', '14', '15', '19', '20', '22', '23']

# event_dates = ['2021-06-24T00:15:00', '2021-06-29T01:29:00', '2021-07-23T15:59:00', '2021-08-03T19:52:00']
full_event_dates = ['2022-02-16T01:46:00', '2022-02-16T05:05:00', '2022-02-16T14:20:00', '2022-02-16T15:50:00', '2021-10-11T16:22:00', '2022-02-16T00:08:00', '2022-02-16T06:09:00', '2022-02-16T08:35:00', '2022-02-16T09:36:00', '2022-02-16T12:58:00', '2022-02-16T14:05:00', '2022-02-16T15:39:00', '2022-02-16T15:56:00', '2022-02-16T17:57:00', '2022-03-01T15:29:00', '2022-04-05T23:10:00', '2022-06-07T14:22:00', '2022-12-28T19:52:00', '2023-01-30T17:55:00', '2021-10-20T19:01:00', '2022-02-16T16:03:00', '2022-03-15T18:12:00', '2022-04-14T14:26:00', '2022-06-14T18:35:00', '2023-02-01T18:22:00', '2021-05-18T20:38:00', '2022-03-17T01:13:00', '2022-05-25T19:45:00', '2022-06-20T16:12:00', '2021-10-29T22:43:00', '2022-03-18T18:46:00', '2022-07-07T20:31:00', '2021-10-07T18:51:00', '2021-11-03T16:28:00', '2022-07-25T19:09:00', '2021-11-05T19:07:00', '2022-08-15T01:55:00', '2021-04-26T17:09:00', '2022-09-06T19:28:00', '2021-11-17T02:00:00', '2022-09-12T10:26:00', '2021-11-17T19:43:00', '2022-09-27T21:49:00', '2022-01-17T15:38:00', '2022-01-24T19:12:00', '2022-11-04T18:31:00', '2022-02-07T00:01:00', '2022-02-12T03:13:00', '2022-02-15T20:01:00', '2022-12-13T21:40:00', '2021-05-12T16:31:00', '2021-06-24T00:15:00', '2021-06-29T01:29:00', '2021-10-02T19:22:00', '2021-07-23T15:59:00', '2021-10-05T19:19:00', '2021-08-03T19:52:00', '2021-08-03T23:45:00', '2021-08-19T15:30:00', '2021-11-05T22:20:00', '2021-04-30T16:40:00']
accelRHs = defaultdict(dict)
for event_date in full_event_dates:
    accelRHs[event_date] = getSensorRH(rawRHs, event_date, channel_numbers)

# Dataset for each channel combination for each model order for each event

In [4]:
import numpy as np
from pandas import DataFrame as df

n_events = 1
n_combo = 10
n_channels = 8
n_time = 20

for e in range(n_events):
    data = np.arange(n_channels*n_time).reshape(n_time, n_channels).T

    for i in range(1,n_combo):
        bin_i = bin(i)
        str_i = str(bin_i)[2:]
        lst_i = list(map(int,str_i))
        idx_i = np.array([0]*(n_channels-len(lst_i)) + lst_i, dtype=bool)

        # print(bin_i)
        # print(str_i)
        # print(lst_i)
        print(idx_i)

[False False False False False False False  True]
[False False False False False False  True False]
[False False False False False False  True  True]
[False False False False False  True False False]
[False False False False False  True False  True]
[False False False False False  True  True False]
[False False False False False  True  True  True]
[False False False False  True False False False]
[False False False False  True False False  True]


In [5]:
print(len(full_event_dates))

61


In [6]:
dt = 0.01
q = 1 # number of input channels
n_modes = 3 # number of modes to get periods for
n_channels = len(output_channel_numbers) # total number of channels
event_dates = full_event_dates[30:] 
n_events = len(event_dates)
n_combo = 256
model_orders = [10, 20, 50]
n_orders = len(model_orders)
periods = np.empty((n_events, n_combo, n_orders, n_modes))

import time
timer_0 = time.perf_counter()
for i_event, event_date in enumerate(event_dates):
    inputRH = accelRHs[event_date]['18']
    window = husid(inputRH, False, dt, lb=0.025, ub=0.9)
    eventRHs = np.array([accelRHs[event_date][ch][window[0]:window[1]] for ch in output_channel_numbers])[:,None]
    for i in range(1,n_combo):
        bin_i = bin(i)
        str_i = str(bin_i)[2:]
        lst_i = list(map(int,str_i))
        idx_i = np.array([0]*(n_channels-len(lst_i)) + lst_i, dtype=bool)
        # print(bin_i)
        # print(str_i)
        # print(lst_i)
        # print(idx_i)
        outputRHarray = eventRHs[idx_i]
        # print(outputRHarray.shape)
        p = len(np.arange(n_channels)[idx_i]) # number of output channels
        for i_r, r in enumerate(model_orders):
            mco = min(300,int(np.floor((outputRHarray.shape[2]-1)/2))) # m_o = m_c = (nt-1)/2
            # print(mco)
            Ar,Br,Cr,Dr,HSVs = okid.era(outputRHarray,mco,mco,p,q,r)
            modes = ExtractModes.modes(dt, Ar, Cr)
            period_i = np.array([1/f for f in sorted([modes[str(i)]['freq'] for i in list(modes.keys())])[:n_modes]])
            periods[i_event, i, i_r, :] = period_i
            print(period_i)
time_to_run = time.perf_counter() - timer_0
print(time_to_run)
with open('periods30-61.dat', 'wb') as f:
    pickle.dump(periods, f)

[1.68008974 0.80733916 0.56254532]
[1.68041844 0.80981822 0.56586879]
[1.7162502  0.8242751  0.56326874]
[0.16055897 0.15975047 0.11752042]
[1.1628154  0.16072878 0.15104039]
[1.04377123 0.30269506 0.27450111]
[0.99426777 0.65803985 0.1600255 ]
[1.70815785 0.81418102 0.58138237]
[1.68084418 1.00812965 0.80493871]
[1.08990096 0.79199854 0.16255336]
[1.65524816 0.76850943 0.58163421]
[1.70232335 0.81439107 0.57703433]
[1.16498712 0.75004593 0.16198193]
[1.6815174  0.80214048 0.57547993]
[1.70513385 0.81179511 0.5696148 ]
[0.77778161 0.16040062 0.13798686]
[1.06545831 0.77814207 0.17371289]
[1.66517663 1.04348722 0.79920827]
[1.0843217  0.73871595 0.15989285]
[1.64519489 0.80575529 0.58196428]
[1.69346922 0.98855184 0.80651834]
[0.56055157 0.16379108 0.14539244]
[0.61547247 0.53211167 0.16442871]
[1.07477884 0.84443027 0.5714296 ]
[0.56295225 0.16924127 0.14532227]
[0.71375938 0.55009724 0.15983358]
[1.68935033 1.25517935 0.80320792]
[0.56099121 0.16122969 0.14551723]
[0.62199775 0.536572

In [7]:
print(np.array(periods).shape)

(31, 256, 3, 3)


In [8]:
414/60

6.9

# Modal Identification from Time History Experiments

In [ ]:
np.array(periods)[5][80][2]

In [ ]:
with open('periods0-5.dat', 'rb') as f:
    periods = pickle.load(f)

print(periods.shape)

In [ ]:
opsperiods = np.loadtxt("./hwd_ops_impulse/PeriodsPostG.txt")[:3]
cgsPds = defaultdict(dict)

In [ ]:
for m in range(3):
    # opsPds[str(m+1)] = [Periods['exp1']['ops'][m], Periods['exp2']['ops'][m], Periods['exp3']['ops'][m], Periods['exp4']['ops'][m]]
    opsPds[str(m+1)] = [Periods['exp1']['ops'][m], Periods['exp2']['ops'][m], Periods['exp4']['ops'][m]]
    # cgsPds[str(m+1)] = [Periods['exp1']['cgs'][m], Periods['exp2']['cgs'][m], Periods['exp3']['cgs'][m], Periods['exp4']['cgs'][m]]
    cgsPds[str(m+1)] = [Periods['exp1']['cgs'][m], Periods['exp2']['cgs'][m], Periods['exp4']['cgs'][m]]

fig, ax = plt.subplots(1,3, figsize=(11,4), tight_layout=True, sharey=True)
ax[0].set_ylim((0,1.6))
ax[0].set_ylabel("Period (s)", fontsize=14)
for m in range(3):
    ax[m].plot(event_dates, opsPds[str(m+1)], '-o', label='numerical ERA prediction')
    ax[m].plot(event_dates, cgsPds[str(m+1)], '--o', label='sensor ERA prediction')
    ax[m].plot(event_dates, Periods['eigen']['ops'][m]*np.ones(3), '--', color='gray', label='numerical eigensolution')
    ax[m].set_xlabel("Event Date", fontsize=14)
    ax[m].set_title(f"Mode {m+1}", fontsize=14)
ax[2].legend(fontsize=11);